In [14]:
import pandas as pd
import time

### File Paths ####
filepath1 = (
 "/Users/pranavsukumaran/Desktop/Personal_dev/netflix/project/data/data.tsv"
)
filepath2 = (
 "/Users/pranavsukumaran/Desktop/Personal_dev/netflix/project/data/data-2.tsv"
)
netflix_path = "/Users/pranavsukumaran/Desktop/Personal_dev/netflix/project/data/processed_data.csv"
output_path = "/Users/pranavsukumaran/Desktop/Personal_dev/netflix/project/data/"
# ----------------------------------------------------------------------------------------#


# Function to load data
def load_imdb(filepath1, filepath2, netflix_path):

 data_rating = pd.read_csv(filepath1, sep="\t")
 data_basic = pd.read_csv(filepath2, sep="\t")
 netflix_data = pd.read_csv(netflix_path)
 return data_rating, data_basic, netflix_data



def merge_and_save(data_basic, data_rating, netflix_data, output_path):
    # Select relevant columns from data_basic
    final_data = data_basic[
        ["tconst", "titleType", "primaryTitle", "isAdult", "startYear", "runtimeMinutes", "genres"]
    ]
    
    # Merge final_data with data_rating
    merged_data = pd.merge(final_data, data_rating, on="tconst")
    
    # Lowercase titles for matching
    merged_data["Title_lower"] = merged_data["primaryTitle"].str.lower()
    netflix_data["Title_lower"] = netflix_data["Title"].str.lower()

    # Map Netflix types to IMDb types
    # Now including 'short' under the 'Movie' category
    def map_to_netflix_type(imdb_type):
        if imdb_type in ['movie', 'short']:
            return 'Movie'
        elif imdb_type in ['tvSeries', 'tvMiniSeries']:
            return 'TV Show'
        return None

    merged_data['Mapped_Type'] = merged_data['titleType'].apply(map_to_netflix_type)
    
    # Merge with Netflix data to ensure matching title types
    type_matched_data = pd.merge(merged_data, netflix_data[['Title_lower', 'Type']], on="Title_lower")
    type_matched_data = type_matched_data[type_matched_data['Mapped_Type'] == type_matched_data['Type']]

    # Selecting title with highest number of votes after ensuring type match
    imdb_max_votes = type_matched_data.loc[
        type_matched_data.groupby("Title_lower")["numVotes"].idxmax()
    ]

    # Final merge with Netflix data
    final_merge = imdb_max_votes.merge(netflix_data, on="Title_lower", how="inner")
    
    # Filtering and dropping unnecessary columns
    final_merge = final_merge[~final_merge["titleType"].isin(["tvMovie", "video", "videoGame"])]
    final_merge = final_merge.drop(["tconst", "isAdult", "runtimeMinutes", "Title_lower", "Title", "Type", "Mapped_Type"], axis=1)
    
    # Saving the final merged data
    final_merge.to_csv(f"{output_path}final_merged_data_1.csv", index=False)



# execution
start_time = time.time()
data_rating, data_basic, netflix_data = load_imdb(filepath1, filepath2, netflix_path)
final_merge = merge_and_save(data_basic, data_rating, netflix_data, output_path)
end_time = time.time()
total_time = end_time - start_time
print(f"Processing time using Pandas: {total_time:.2f} seconds")
print(" ")
print("---------------------------------------------------------")
print(" ")
print("Final Processed Dataset:")
print(" ")


/var/folders/fk/b7dd25t91rq06z3l8zsn15r00000gn/T/ipykernel_69407/279902178.py:20: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data_basic = pd.read_csv(filepath2, sep="\t")


KeyError: "['Type'] not found in axis"

In [6]:
print("Columns in data_basic:", data_basic.columns.tolist())
print("Columns in data_rating:", data_rating.columns.tolist())
print("Columns in netflix_data:", netflix_data.columns.tolist())


Columns in data_basic: ['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres']
Columns in data_rating: ['tconst', 'averageRating', 'numVotes']
Columns in netflix_data: ['Profile Name', 'Duration', 'Title', 'Country', 'Type', 'Hour', 'Day', 'Month', 'Year', 'Device Category', 'Title_lower']
